In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score, accuracy_score
from sklearn.linear_model import LinearRegression, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import LeavePOut
from xgboost import XGBClassifier

## 데이터 전처리

In [3]:
df_jinhyeok = pd.read_csv("./data/진혁_데이터프레임.csv")
df_jinhyeok.rename(columns={"Unnamed: 0":"행정구역","Unnamed: 1":"계절","Unnamed: 2":"연도"},inplace=True)

month_spring = ["_3","_4","_5"]           # ["_3","_4","_5"]
month_summer = ["_6","_7","_8"]      # ["_5","_6","_7","_8"]
month_fall = ["_8","_9","_10"]      # ["_8","_9","_10","_11"]
month_winter = ["_9","_10","_11"]      # ["_8","_9","_10","_11","_12","_1","_2"]

season = ["spring", "summer", "fall", "winter"]

for i in range(len(season)) : 
    globals()["df_"+season[i]] = df_jinhyeok[df_jinhyeok["계절"]==season[i]]  # 계절별로 데이터 프레임 형성
    globals()["df_"+season[i]] = globals()["df_"+season[i]].groupby(["행정구역","계절","연도"]).mean()   # 행정구역, 계절, 연도를 인덱스로 groupby

In [4]:
spr = "|".join(w for w in month_spring)

In [5]:
for i in range(len(season)) : 
    globals()["month_str_"+season[i]] = "|".join(w for w in globals()["month_"+season[i]])  # 각 계절별 월을 | 으로 join
    
    # 각 계절이 속하는 월과 면적, 생산이 속하는 컬럼명을 리스트로 저장
    globals()["col_of_"+season[i]] = globals()["df_"+season[i]].columns[globals()["df_"+season[i]].columns.str.contains(globals()["month_str_"+season[i]]+"|면적|생산")]

In [6]:
# 데이터프레임을 계절별로 분리
df_spring = df_spring[col_of_spring]
df_summer = df_summer[col_of_summer]
df_fall = df_fall[col_of_fall]
df_winter = df_winter[col_of_winter]

In [7]:
df_winter.dropna(how="any",inplace=True)
# df_winter

In [8]:
city = ["서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시", "세종특별자치시", "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]
city = sorted(city)
# city

In [9]:
# df_winter[df_winter["생산량 (톤)"].isnull()]

In [10]:
# 생산량이 NaN인 행을 drop
df_spring.dropna(how="any", inplace=True)
df_summer.dropna(how="any", inplace=True)
df_fall.dropna(how="any", inplace=True)

In [11]:
# 면적 컬럼 삭제
for s in season : 
    globals()["df_"+s].drop(columns="면적 (ha)", inplace=True)

In [12]:
# 배추 생산 시기를 임의로 상, 중, 하로 나누었으며, 거기에는 육묘기, 생육기, 결구기 등이 포함돼 있음
for s in season : 
    globals()["df_"+s].columns = ['평균기온(°C)_상', '최고기온(°C)_상', '최저기온(°C)_상', '월합강수량(00~24h만)(mm)_상',
       '합계 일사량(MJ/m2)_상', '평균기온(°C)_중', '최고기온(°C)_중', '최저기온(°C)_중',
       '월합강수량(00~24h만)(mm)_중', '합계 일사량(MJ/m2)_중', '평균기온(°C)_하',
       '최고기온(°C)_하', '최저기온(°C)_하', '월합강수량(00~24h만)(mm)_하',
       '합계 일사량(MJ/m2)_하', '생산량 (톤)']

In [13]:
df_total = pd.concat([df_spring, df_summer, df_fall, df_winter])
df_total.head()

평균기온(°C)_상  최고기온(°C)_상  최저기온(°C)_상  월합강수량(00~24h만)(mm)_상  \
행정구역 계절     연도                                                               
강원도  spring 2000        4.40       19.30        -8.6                 14.10   
            2001        5.25       19.75        -8.6                  2.55   
            2002        7.30       19.70        -7.3                 33.10   
            2003        6.60       21.30        -6.0                 28.50   
            2004        6.90       20.30        -4.9                 19.90   

                  합계 일사량(MJ/m2)_상  평균기온(°C)_중  최고기온(°C)_중  최저기온(°C)_중  \
행정구역 계절     연도                                                          
강원도  spring 2000           448.65       10.40        23.7        -3.1   
            2001           455.28       10.85        23.7        -2.2   
            2002           519.21       12.70        24.8        -1.2   
            2003           523.00       11.90        25.9        -1.4   
            2004           481.75       12.60        25.4        -0.5   

                  월합강수량(00~24h만)(mm)_중  합계 일사량(MJ/m2)_중  평균기온(°C)_하  \
행정구역 계절     연도                                                        
강원도  spring 2000                 37.10          486.355       15.90   
            2001                 23.55          530.950       17.05   
            2002                 50.00          542.480       17.70   
            2003                 49.00          540.765       17.50   
            2004                 16.30          537.210       17.80   

                  최고기온(°C)_하  최저기온(°C)_하  월합강수량(00~24h만)(mm)_하  \
행정구역 계절     연도                                                   
강원도  spring 2000        31.6        3.90                 75.20   
            2001        31.4        5.55                 67.55   
            2002        34.1        5.70                 65.00   
            2003        34.5        5.60                 52.60   
            2004        33.9        5.80                 47.70   

                  합계 일사량(MJ/m2)_하   생산량 (톤)  
행정구역 계절     연도                               
강원도  spring 2000          520.280   87619.0  
            2001          562.770   83405.0  
            2002          586.320   56908.0  
            2003          586.195   82760.0  
            2004          540.490  100486.0

In [14]:
# df_total[df_total["생산량 (톤)"] == 0]

In [15]:
# 생산량이 0이상인 지역의 기후 조건을 적합, 0인 지역의 기후 조건을 부적합하다고 labeling
df_total["재배 적합"] = df_total["생산량 (톤)"].apply(lambda x : 1 if x > 0 else 0)

In [16]:
df_total[df_total["재배 적합"] == 0]
# df_total.head()

평균기온(°C)_상  최고기온(°C)_상  최저기온(°C)_상  월합강수량(00~24h만)(mm)_상  \
행정구역  계절     연도                                                               
대전광역시 spring 2012        7.30        19.8       -1.70                 53.90   
             2014        8.50        19.3       -2.20                 47.90   
             2015        6.00        17.4       -0.50                 45.80   
서울특별시 spring 2010        4.10        19.2       -5.50                 88.60   
             2011        3.20        18.7       -9.00                 26.70   
...                       ...         ...         ...                   ...   
충청북도  winter 2017       22.65        30.0       14.70                112.70   
             2018       22.10        30.2       13.50                139.90   
             2019       24.40        30.7       18.85                462.40   
             2020       21.75        29.7       13.15                210.95   
             2021       20.20        29.0       12.60                185.00   

                   합계 일사량(MJ/m2)_상  평균기온(°C)_중  최고기온(°C)_중  최저기온(°C)_중  \
행정구역  계절     연도                                                          
대전광역시 spring 2012          390.280       14.10       28.00        3.80   
             2014          427.520       13.80       28.40        3.20   
             2015          430.880       12.30       24.80        4.60   
서울특별시 spring 2010          343.625        9.20       24.80       -3.70   
             2011          506.250       10.10       22.60       -1.20   
...                            ...         ...         ...         ...   
충청북도  winter 2017          380.610       16.95       29.50        2.60   
             2018          415.265       15.60       30.60        0.90   
             2019          374.920       20.00       29.25        9.45   
             2020          440.290       14.95       29.75        0.30   
             2021          426.565       13.50       29.50       -1.70   

                   월합강수량(00~24h만)(mm)_중  합계 일사량(MJ/m2)_중  평균기온(°C)_하  \
행정구역  계절     연도                                                        
대전광역시 spring 2012                 38.50          496.970       17.70   
             2014                 78.20          443.545       17.70   
             2015                 48.50          523.180       15.70   
서울특별시 spring 2010                 70.50          494.850       16.70   
             2011                133.00          498.130       15.90   
...                                 ...              ...         ...   
충청북도  winter 2017                 29.50          433.200        9.95   
             2018                 43.20          423.275        8.90   
             2019                 32.75          409.190       14.05   
             2020                 42.80          388.865        7.95   
             2021                 33.80          407.200        6.40   

                   최고기온(°C)_하  최저기온(°C)_하  월합강수량(00~24h만)(mm)_하  \
행정구역  계절     연도                                                   
대전광역시 spring 2012       28.50        8.10                 97.70   
             2014       30.40        8.90                121.60   
             2015       27.30        8.10                 63.90   
서울특별시 spring 2010       30.80        1.50                110.00   
             2011       29.10        6.00                104.50   
...                       ...         ...                   ...   
충청북도  winter 2017       21.95       -1.45                 51.95   
             2018       22.60       -2.80                 86.40   
             2019       22.50        6.60                 59.40   
             2020       21.85       -3.75                 89.50   
             2021       19.90       -5.20                 44.90   

                   합계 일사량(MJ/m2)_하  생산량 (톤)  재배 적합  
행정구역  계절     연도                                     
대전광역시 spring 2012          560.500      0.0      0  
             2014          527.260      0.0      0  
       

In [37]:
df_spring[df_spring["생산량 (톤)"]==0]

평균기온(°C)_상  최고기온(°C)_상  최저기온(°C)_상  월합강수량(00~24h만)(mm)_상  \
행정구역    계절     연도                                                               
대전광역시   spring 2012        7.30       19.80       -1.70                  53.9   
               2014        8.50       19.30       -2.20                  47.9   
               2015        6.00       17.40       -0.50                  45.8   
서울특별시   spring 2010        4.10       19.20       -5.50                  88.6   
               2011        3.20       18.70       -9.00                  26.7   
               2013        6.30       22.25       -4.95                  23.1   
               2014        4.75       20.80       -5.85                  16.0   
               2015        5.30       21.00       -4.10                  23.1   
세종특별자치시 spring 2015        4.90       20.20       -7.50                  79.0   
               2017        3.90       13.80       -4.90                  26.6   
인천광역시   spring 2014        5.60       22.50       -7.70                   9.5   

                     합계 일사량(MJ/m2)_상  평균기온(°C)_중  최고기온(°C)_중  최저기온(°C)_중  \
행정구역    계절     연도                                                          
대전광역시   spring 2012          390.280        14.1       28.00        3.80   
               2014          427.520        13.8       28.40        3.20   
               2015          430.880        12.3       24.80        4.60   
서울특별시   spring 2010          343.625         9.2       24.80       -3.70   
               2011          506.250        10.1       22.60       -1.20   
               2013          559.580        12.6       24.60        0.70   
               2014          552.965        11.3       26.85       -1.45   
               2015          560.620        12.2       25.60        1.80   
세종특별자치시 spring 2015          408.860        12.5       29.10       -1.90   
               2017          416.010        10.6       28.90        0.50   
인천광역시   spring 2014          410.180        12.7       27.50        0.80   

                     월합강수량(00~24h만)(mm)_중  합계 일사량(MJ/m2)_중  평균기온(°C)_하  \
행정구역    계절     연도                                                        
대전광역시   spring 2012                 38.50          496.970        17.7   
               2014                 78.20          443.545        17.7   
               2015                 48.50          523.180        15.7   
서울특별시   spring 2010                 70.50          494.850        16.7   
               2011                133.00          498.130        15.9   
               2013                127.25          579.325        17.9   
               2014                 83.10          533.500        16.7   
               2015                103.00          587.230        19.0   
세종특별자치시 spring 2015                 92.40          519.835        17.7   
               2017                104.50          505.070        17.2   
인천광역시   spring 2014                 96.10          391.210        18.3   

                     최고기온(°C)_하  최저기온(°C)_하  월합강수량(00~24h만)(mm)_하  \
행정구역    계절     연도                                                   
대전광역시   spring 2012       28.50        8.10                 97.70   
               2014       30.40        8.90                121.60   
               2015       27.30        8.10                 63.90   
서울특별시   spring 2010       30.80        1.50                110.00   
               2011       29.10        6.00                104.50   
               2013       28.25        8.35                137.05   
               2014       28.75        6.35                146.95   
               2015       30.60        8.40                142.90   
세종특별자치시 spring 2015       29.00        7.00                 40.90   
               2017       27.40        7.10                 10.00   
인천광역시   spring 2014       31.80        5.10                 32.60   

                     합계 일사량(MJ/m2)_하  생산량 (톤)  
행정구역    계절     연도                              
대전광역시   spring 2012  

In [17]:
df_total.drop(columns="생산량 (톤)", inplace=True)
df_total.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1212 entries, ('강원도', 'spring', 2000) to ('충청북도', 'winter', 2021)
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   평균기온(°C)_상            1212 non-null   float64
 1   최고기온(°C)_상            1212 non-null   float64
 2   최저기온(°C)_상            1212 non-null   float64
 3   월합강수량(00~24h만)(mm)_상  1212 non-null   float64
 4   합계 일사량(MJ/m2)_상       1212 non-null   float64
 5   평균기온(°C)_중            1212 non-null   float64
 6   최고기온(°C)_중            1212 non-null   float64
 7   최저기온(°C)_중            1212 non-null   float64
 8   월합강수량(00~24h만)(mm)_중  1212 non-null   float64
 9   합계 일사량(MJ/m2)_중       1212 non-null   float64
 10  평균기온(°C)_하            1212 non-null   float64
 11  최고기온(°C)_하            1212 non-null   float64
 12  최저기온(°C)_하            1212 non-null   float64
 13  월합강수량(00~24h만)(mm)_하  1212 non-null   float64
 14  합계 일사량(MJ/m2)_하       1212 non

In [18]:
# 상관 분석(안 해도 되나)
# plt.rc('font', family='Malgun Gothic')

# corr = np.corrcoef(df_total.values.T)
# fig = plt.figure(figsize=(16,16))
# sns.heatmap(corr, cbar=True, annot=True, square=False, xticklabels=df_total.columns, yticklabels=df_total.columns, annot_kws={"size":7})

## train, test set 분리

In [19]:
X = df_total.iloc[:,:-1]
y = df_total.iloc[:,-1:]

In [20]:
std = StandardScaler()
std.fit(X)
X = std.transform(X)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

## GridSearchCV를 통한 최적의 파라미터

In [52]:
# 교차 검증을 위한 kfold
kfold = KFold(n_splits=10, shuffle=True, random_state=27)
skfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=27)

alpha = np.logspace(-4,2,7)   # lasso, ridge 해당
learning_rate = np.logspace(-4,2,7)   # adaboost, gradientboost 해당
eta = np.logspace(-4,2,7)   # xgboost 해당
max_iter = [1,3,5,10,20,50,100,150,200]   # lasso, ridge 해당
solver = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']   # ridge 해당, 계산에 사용할 알고리즘
n_estimators = [1,3,5,10,15,20,50,100,200,500]   # randomforesteregressor(사용할 tree의 개수), gradientboost, adaboost, xgboost 해당
max_depth = [2,3,5,7,10,12]  # randomforesteregressor(tree의 깊이), gradientboost, xgboost 해당
criterion_rf = ["gini", "entropy", "log_loss"]  # randomforesteregressor 해당
criterion_gb = ["squared_error", "friedman_mse"]  # gradientboost 해당
algorithm_ada = ["SAMME", "SAMME.R"]    # adaboost 해당
loss_gb = ["log_loss", "deviance", "exponential"]  # gradientboost 해당
gamma = [0, 0.01, 0.05, 0.1, 0.2, 0.5]    # xgboost 해당
max_leaf_nodes = [2,3,5,7]     # randomforest, xgboost 해당
min_child_weight = [1,2,3,4,5]   # xgboost 해당
min_samples_leaf = [1,2,3,4,5]   # randomforesst 해당
min_weight_fraction_leaf = [0, 0.1, 0.2, 0.4, 0.5]  # randomforest 해당
max_features = ["sqrt", "log2"]   # randomforest 해당
max_features_grad = ["auto", "sqrt", "log2"]   # randomforest 해당

params_rf = {"n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "criterion" : criterion_rf,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_samples_leaf" : min_samples_leaf,
                "min_weight_fraction_leaf" : min_weight_fraction_leaf,
                "max_features" : max_features}

params_ada = {"learning_rate" : learning_rate, 
                "n_estimators" : n_estimators, 
                "algorithm" : algorithm_ada
                }

params_grad = {"learning_rate" : learning_rate, 
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth,
                "criterion" : criterion_gb,
                "loss" : loss_gb,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_samples_leaf" : min_samples_leaf,
                "min_weight_fraction_leaf" : min_weight_fraction_leaf,
                "max_features" : max_features_grad}

params_xgb = {"learning_rate" : eta,
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "gamma" : gamma,
                "max_leaf_nodes" : max_leaf_nodes,
                "min_child_weight" : min_child_weight}



In [53]:
rf = RandomForestClassifier()
grid_rf = GridSearchCV(estimator=rf, param_grid=params_rf, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_rf.fit(X, y)
print("Best Score :", grid_rf.best_score_)
print("Best Parameters :", grid_rf.best_params_)

KeyboardInterrupt: 

In [44]:
ada = AdaBoostClassifier()
grid_ada = GridSearchCV(estimator=ada, param_grid=params_ada, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_ada.fit(X, y)
print("Best Score :", grid_ada.best_score_)
print("Best Parameters :", grid_ada.best_params_)

Best Score : -0.11474055006096735
Best Parameters : {'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 100}


In [45]:
grad = GradientBoostingClassifier()
grid_grad = GridSearchCV(estimator=grad, param_grid=params_grad, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_grad.fit(X, y)
print("Best Score :", grid_grad.best_score_)
print("Best Parameters :", grid_grad.best_params_)

Best Score : -0.08828749491938762
Best Parameters : {'criterion': 'squared_error', 'learning_rate': 1.0, 'loss': 'exponential', 'max_depth': 5, 'n_estimators': 100}


In [46]:
xgb = XGBClassifier()
grid_xgb = GridSearchCV(estimator=xgb, param_grid=params_xgb, cv=skfold, n_jobs=-1, scoring="neg_mean_squared_error")
grid_xgb.fit(X, y)
print("Best Score :", grid_xgb.best_score_)
print("Best Parameters :", grid_xgb.best_params_)

Best Score : -0.0924129521745021
Best Parameters : {'eta': 0.1, 'max_depth': 5, 'n_estimators': 50}


## logistic regression을 통한 이진분류

In [22]:
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

print("accuracy :", accuracy_score(y_test, y_pred))

accuracy : 0.7366255144032922


In [23]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print("accuracy :", accuracy_score(y_test, y_pred))

accuracy : 0.8847736625514403


In [24]:
# df_summer

In [34]:
test = [15.90,21.80,12.20,133.50,471.260,17.80,25.10,18.40,384.00,529.200,20.7,26.60,19.70,415.00,463.430]
test_array = np.array(test).reshape(1,-1)

In [35]:
rf.predict(test_array)

array([0], dtype=int64)